In [148]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "mnist"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:5000"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

#from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

In [203]:
from __future__ import annotations

import os
from pathlib import Path
from posixpath import join as urljoin
from typing import Any
from urllib.parse import urlparse, urlunparse
import inspect
import requests
from typing import Any, cast

DEBUG = True
class Session(object):
    pass

def create_data_dict(**kwargs):
    return kwargs

def debug(url, data=None):
    if DEBUG:
        print("url:", url)
        if (data):
            print("data:", data)
def get(session, endpoint, *features):
    debug(urljoin(endpoint, *features))
    try:
        return session.get(urljoin(endpoint, *features)).json()
    except requests.ConnectionError as e:
        handle_error(e)
    except Exception as e:
        handle_error(e)
def post(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    try:
        return session.post(urljoin(endpoint, *features), data=data).json()
    except Exception as e:
        return session.post(urljoin(endpoint, *features), data=data).text
def delete(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    try:
        ret = cast(dict[str, Any], session.post(urljoin(endpoint, *features), data=data).json())
    except requests.ConnectionError as e:
        handle_error(e)
    except Exception as e:
        handle_error(e)
def put(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    return make_request(session, 'put', endpoint, data, *features)
    try:
        return cast(dict[str, Any], session.post(urljoin(endpoint, *features), data=data).json())
    except requests.ConnectionError as e:
        handle_error(e)
    except Exception as e:
        handle_error(e)
def make_request(session, method, endpoint, data, *features)
    url = urljoin(endpoint, *features)
    method = getattr(session, method)
    return method(url, data=data).json()
def handle_error(error):
    print(error)

class DioptraClient(object):
    def __init__(self, session, address=None, api_version="v1") -> None:
        self._session = session
        self._users = UsersClient(session, "users", address, api_version)
        self._auth = AuthClient(session, "auth", address, api_version)
    @property
    def users(self):
        return self.get_endpoint(self._users)
    @property
    def auth(self):
        return self.get_endpoint(self._auth)

    def get_endpoint(self, ep):
        ep.session = self._session
        return ep

class Endpoint(object):
    def __init__(self, session, ep_name, address, api_version) -> None:
        address = (
            f"{address}/api/{api_version}"
            if address
            else f"{os.environ['DIOPTRA_RESTAPI_URI']}/api/{api_version}"
        )
        self._scheme, self._netloc, self._path, _, _, _ = urlparse(address)
        self._ep_name = ep_name
        self._session = session
    @property
    def session(self):
        return self._session
    @session.setter
    def session(self, s):
        self._session = s
    @property
    def url(self):
        return self.def_endpoint(self._ep_name)
    def def_endpoint(self, name):
        '''creates base url for an endpoint by name'''
        return urlunparse(
            (self._scheme, self._netloc, urljoin(self._path, name + "/"), "", "", "")
        )

class UsersClient(Endpoint):
    def get_all(self):
        '''gets all users'''
        return get(self.session, self.url)
    def create(self, username, email, password, confirm_password):
        '''creates a user'''
        d = {"username":username,
             "email": email,
             "password": password,
             "confirm_password": confirm_password}
        return post(self.session, self.url, d, '')
    def get_by_id(self, id):
        '''get a user by id'''
        return get(self.session, self.url, id)
    def update_password_by_id(self, id, old_password, new_password, confirm_new_password):
        '''change a user's password by id'''
        d = {"old_password":old_password,
             "new_password": new_password,
             "confirm_new_password": confirm_new_password}
        return post(self.session, self.url, d, id)
    def current(self):
        '''get the current user'''
        return get(self.session, self.url, 'current')
    def delete_current(self, password):
        '''delete the current user'''
        d = {"password":password}
        return delete(self.session, self.url, d, 'current')
    def modify_current(self, username, email):
        '''modify the current user'''
        d = {"username":username,
             "email": email}
        return put(self.session, self.url, d, 'current')
    def modify_current_password(self, old_password, new_password, confirm_new_password):
        '''modify the current user's password'''
        d = {"old_password":old_password,
             "new_password": new_password,
             "confirm_new_password": confirm_new_password}
        return put(self.session, self.url, d, 'current', 'password')

class AuthClient(Endpoint):
    def login(self, username, password):
        d = {"username":username,
             "password": password}
        return post(self.session, self.url, d, 'login')
    def logout(self, everywhere):
        d = {"everywhere": everywhere}
        return post(self.session, self.url, d, 'logout')

In [204]:
client = DioptraClient(requests.Session())
client.users.get_by_id('1234')
client.users.create('a','a@a.com','b','b')

url: http://localhost:5000/api/v1/users/1234
url: http://localhost:5000/api/v1/users/
data: {'username': 'a', 'email': 'a@a.com', 'password': 'b', 'confirm_password': 'b'}


{'message': 'The browser (or proxy) sent a request that this server could not understand.'}

In [205]:
client.auth.login('dioptra','dioptra')

url: http://localhost:5000/api/v1/auth/login
data: {'username': 'dioptra', 'password': 'dioptra'}


{'message': 'The browser (or proxy) sent a request that this server could not understand.'}

In [206]:
client.users.get_all()

url: http://localhost:5000/api/v1/users/


{'message': "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}

In [207]:
client.users.create('a','a@a.com','b','b')

url: http://localhost:5000/api/v1/users/
data: {'username': 'a', 'email': 'a@a.com', 'password': 'b', 'confirm_password': 'b'}


{'message': 'The browser (or proxy) sent a request that this server could not understand.'}

In [208]:
client.users.get_by_id('1234')

url: http://localhost:5000/api/v1/users/1234


{'message': "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}

In [209]:
client.users.update_password_by_id('1234','oldp','newp','newp')

url: http://localhost:5000/api/v1/users/1234
data: {'old_password': 'oldp', 'new_password': 'newp', 'confirm_new_password': 'newp'}


{'message': 'The method is not allowed for the requested URL.'}

In [210]:
client.users.current()

url: http://localhost:5000/api/v1/users/current


{'message': "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}

In [211]:
client.users.delete_current('myp')

url: http://localhost:5000/api/v1/users/current
data: {'password': 'myp'}


In [212]:
client.users.modify_current('newuser','newemail@email.com')

url: http://localhost:5000/api/v1/users/current
data: {'username': 'newuser', 'email': 'newemail@email.com'}


In [213]:
client.users.modify_current_password('oldp','newp','newp')

url: http://localhost:5000/api/v1/users/current/password
data: {'old_password': 'oldp', 'new_password': 'newp', 'confirm_new_password': 'newp'}


In [214]:
client.auth.logout(True)

url: http://localhost:5000/api/v1/auth/logout
data: {'everywhere': True}


{'message': "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}